In [5]:
import sys
import os
sys.path.append('../../')

%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../../data')

train_df = shuffle(train_df)
valid_df = shuffle(valid_df)

print(train_df.head())


                                                  image                   id  \
2473  ../../data/Train/CameraRGB/episode_0011_000061...  episode_0011_000061   
4502  ../../data/Train/CameraRGB/episode_0000_000055...  episode_0000_000055   
3149                 ../../data/Train/CameraRGB/702.png                  702   
1844  ../../data/Train/CameraRGB/episode_0004_000184...  episode_0004_000184   
3800  ../../data/Train/CameraRGB/episode_0011_000138...  episode_0011_000138   

                                                  label  
2473  ../../data/Train/CameraSeg/episode_0011_000061...  
4502  ../../data/Train/CameraSeg/episode_0000_000055...  
3149                 ../../data/Train/CameraSeg/702.png  
1844  ../../data/Train/CameraSeg/episode_0004_000184...  
3800  ../../data/Train/CameraSeg/episode_0011_000138...  


In [4]:
from models.unet import model_unetVGG16

model = model_unetVGG16(12, image_shape=(320, 416, 3), keep_prob=0.5)
model.summary()

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 416, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 320, 416, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 320, 416, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 160, 208, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [7]:
from gen.datagen import balanced_generator_from_df

BATCH_SIZE = 32

model_dir = '../../saved_models/unet/unet_v2/'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)


train_gen = balanced_generator_from_df(train_df, BATCH_SIZE, (320, 416))
valid_gen = balanced_generator_from_df(valid_df, BATCH_SIZE, (320, 416))


In [8]:
from train import euclidean_distance_loss, jaccard_distance_loss
from keras.utils.training_utils import multi_gpu_model
from keras.optimizers import RMSprop, Adam

gpus = 2

model_gpu = multi_gpu_model(model, gpus)
# ad = Adam(lr=1e-4)

model_gpu.compile(loss=jaccard_distance_loss,
              optimizer='rmsprop',
              metrics=['acc'])


In [9]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model_gpu, 
                   train_gen, 
                   valid_gen, 
                   training_size=m, 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=100,
                  gpus = 2)

Instructions for updating:
Use the retry module or similar alternatives.
Epoch 1/100
75/75 [==============================] - 89s 1s/step - loss: 1.3290 - acc: 0.3231 - val_loss: 1.3859 - val_acc: 0.2932

Epoch 00001: val_loss improved from inf to 1.38589, saving model to ../../saved_models/unet/unet_v2//model.hdf5
Epoch 2/100
75/75 [==============================] - 83s 1s/step - loss: 1.0602 - acc: 0.4643 - val_loss: 0.4612 - val_acc: 0.7669

Epoch 00002: val_loss improved from 1.38589 to 0.46122, saving model to ../../saved_models/unet/unet_v2//model.hdf5
Epoch 3/100
75/75 [==============================] - 85s 1s/step - loss: 0.3964 - acc: 0.7987 - val_loss: 0.3450 - val_acc: 0.8251

Epoch 00003: val_loss improved from 0.46122 to 0.34500, saving model to ../../saved_models/unet/unet_v2//model.hdf5
Epoch 4/100
75/75 [==============================] - 84s 1s/step - loss: 0.2793 - acc: 0.8581 - val_loss: 0.3241 - val_acc: 0.8355

Epoch 00004: val_loss improved from 0.34500 to 0.32410,

KeyboardInterrupt: 

In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [10]:
# model_gpu.load_weights('../../saved_models/unet/unet_v1/model.hdf5')
model.save('../../saved_models/unet/unet_v2/model_saved.h5')